## Bayesian Optimisation for Antimicrobial Polymer Discovery

In [191]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
import torch
from botorch.models import SingleTaskGP, ModelListGP
from botorch.models.gp_regression_mixed import MixedSingleTaskGP
from botorch.fit import fit_gpytorch_model
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition.monte_carlo import qExpectedImprovement, qUpperConfidenceBound
from botorch.acquisition.analytic import UpperConfidenceBound, ProbabilityOfImprovement, ExpectedImprovement
from botorch.optim import optimize_acqf, optimize_acqf_mixed
from botorch.cross_validation import gen_loo_cv_folds
import math
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc

## Retrieve Training dataset and assign variables
To enable mixed search space, utilise MixedSingleTaskGP which uses a special kernel to combine continuous and categorical data.

In [135]:
data = pd.read_excel('dataset_final.xlsx', sheet_name = 'Dataset_Complete_modified')
data

,Polymer Index,type_A,type_B1,type_B2,type_C,composition_A,composition_B1,composition_B2,composition_C,block_sequence_theoretical,...,A2,B2,C2,A3,B3,C3,A4,B4,C4,cLogP_predicted
0,1,Boc-AEAm,PEAm,None,HEAm,0.5,0.30,0.00,0.20,ABC,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.67630
1,2,Boc-AEAm,PEAm,None,HEAm,0.5,0.30,0.00,0.20,ABC,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.67630
2,3,Boc-AEAm,PEAm,None,HEAm,0.5,0.30,0.00,0.20,ABC,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.67630
3,4,Boc-AEAm,PEAm,None,None,0.7,0.30,0.00,0.00,AB,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.05270
4,5,Boc-AEAm,PEAm,None,None,0.7,0.30,0.00,0.00,AB,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.05270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,157,AAPTAC,PEAm,NIPAm,HEAm,0.3,0.00,0.47,0.23,ABC,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,-1.33796
153,158,AAPTAC,PEAm,NIPAm,HEAm,0.3,0.70,0.00,0.00,ABC,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,-0.16180
154,159,AAPTAC,PEAm,NIPAm,HEAm,0.3,0.47,0.23,0.00,ABC,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,-0.45137
155,160,AAPTAC,PEAm,NIPAm,HEAm,0.3,0.23,0.47,0.00,ABC,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,-0.75353


In [136]:
data.corr()

/var/folders/dh/dgy1v49j4rs7lcfxbxnn5bzm0000gn/T/ipykernel_13211/1185648202.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.corr()


,Polymer Index,composition_A,composition_B1,composition_B2,composition_C,Number of blocks,dpn,Target,NMR,GPC,...,A2,B2,C2,A3,B3,C3,A4,B4,C4,cLogP_predicted
Polymer Index,1.000000,-0.237395,-0.209524,0.402446,0.045304,-0.515925,-0.249226,-0.410508,-0.556035,-0.257679,...,-0.243827,-0.438501,-0.387557,-0.226511,-0.190109,-0.172808,NaN,-0.151171,-0.151171,-0.679035
composition_A,-0.237395,1.000000,-0.282589,-0.310371,-0.412450,-0.010745,-0.004903,0.113958,0.207758,-0.182915,...,0.045402,-0.031455,-0.029569,-0.016070,-0.015225,-0.013064,NaN,-0.011290,-0.011290,-0.086684
composition_B1,-0.209524,-0.282589,1.000000,-0.399126,-0.309069,0.271207,0.120026,0.257981,0.273500,0.010997,...,0.129752,0.221584,0.180416,0.118880,0.112632,0.096645,NaN,0.083517,0.083517,0.364220
composition_B2,0.402446,-0.310371,-0.399126,1.000000,-0.286314,-0.275958,-0.117630,-0.274649,-0.471571,-0.175124,...,-0.127162,-0.228049,-0.214375,-0.116507,-0.110385,-0.094716,NaN,-0.081850,-0.081850,-0.102905
composition_C,0.045304,-0.412450,-0.309069,-0.286314,1.000000,0.016395,0.002952,-0.096356,-0.010390,0.344606,...,-0.047429,0.038591,0.063775,0.014118,0.013376,0.011478,NaN,0.009919,0.009919,-0.170816
Number of blocks,-0.515925,-0.010745,0.271207,-0.275958,0.016395,1.000000,0.280251,0.386098,0.408109,0.324969,...,0.452140,0.623867,0.595649,0.632507,0.467679,0.401295,NaN,0.541927,0.541927,0.222334
dpn,-0.249226,-0.004903,0.120026,-0.117630,0.002952,0.280251,1.000000,0.884207,0.766664,0.784606,...,0.114781,0.265455,0.222880,0.203834,0.084057,0.118917,NaN,0.062328,0.062328,0.096098
Target,-0.410508,0.113958,0.257981,-0.274649,-0.096356,0.386098,0.884207,1.000000,0.889259,0.813675,...,0.128931,0.297960,0.256310,0.312228,0.089902,0.116247,NaN,0.236861,0.236861,0.118706
NMR,-0.556035,0.207758,0.273500,-0.471571,-0.010390,0.408109,0.766664,0.889259,1.000000,0.679494,...,0.141521,0.343140,0.223529,0.312518,0.110646,0.178843,NaN,0.162049,0.162049,0.229514
GPC,-0.257679,-0.182915,0.010997,-0.175124,0.344606,0.324969,0.784606,0.813675,0.679494,1.000000,...,0.075559,0.213651,0.229229,0.319477,0.074642,0.083392,NaN,0.294095,0.294095,-0.272896


In [137]:
data = data.drop(columns=['composition_C','Polymer Index','Dispersity','clogP','block_sequence_theoretical', 'block_sequence_experimental','MIC_PAO1', 'MIC_PA','MIC_EC', 'MIC_AB', 'MIC_SA', 'MIC_MSmeg','GPC','Target','NMR'])
data = data.replace({'>128':128,'>256':256, '32-64':64, '64-128':128,'128-256':256})

## Assign Classes

In [138]:
data['Category'] = data['MIC_PAO1_PA'].apply(lambda x: 1 if x <= 64 else 0)
data = data.drop(columns = ['MIC_PAO1_PA'])

In [140]:
data_with_dummies = pd.get_dummies(data, drop_first=True)
data_with_dummies

,composition_A,composition_B1,composition_B2,Number of blocks,dpn,A1,B1,C1,A2,B2,...,C4,cLogP_predicted,Category,type_A_Boc-AEAm,type_A_DMAEA,type_B1_PEAm,type_B2_None,type_C_HEAm,type_C_None,type_C_PEGA
0,0.5,0.30,0.00,1,100,0.3330,0.3330,0.334,0.0,0.0,...,0.0,0.67630,1,1,0,1,1,1,0,0
1,0.5,0.30,0.00,1,40,0.3325,0.3325,0.335,0.0,0.0,...,0.0,0.67630,1,1,0,1,1,1,0,0
2,0.5,0.30,0.00,1,20,0.3300,0.3350,0.335,0.0,0.0,...,0.0,0.67630,1,1,0,1,1,1,0,0
3,0.7,0.30,0.00,1,100,0.5000,0.5000,0.000,0.0,0.0,...,0.0,1.05270,0,1,0,1,1,0,1,0
4,0.7,0.30,0.00,1,40,0.5000,0.5000,0.000,0.0,0.0,...,0.0,1.05270,1,1,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,0.3,0.00,0.47,1,40,0.3325,0.3325,0.335,0.0,0.0,...,0.0,-1.33796,0,0,0,1,0,1,0,0
153,0.3,0.70,0.00,1,40,0.3325,0.3325,0.335,0.0,0.0,...,0.0,-0.16180,0,0,0,1,0,1,0,0
154,0.3,0.47,0.23,1,40,0.3325,0.3325,0.335,0.0,0.0,...,0.0,-0.45137,0,0,0,1,0,1,0,0
155,0.3,0.23,0.47,1,40,0.3325,0.3325,0.335,0.0,0.0,...,0.0,-0.75353,0,0,0,1,0,1,0,0


In [146]:
data_with_dummies.to_csv('modified_data.csv', index=False)

In [147]:
Y_train = data_with_dummies['Category']
X = data_with_dummies.drop(columns = ['Category'])

## Standardise dataset

In [148]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [149]:
X_scaled = scaler.fit_transform(X)
X_train = pd.DataFrame(X_scaled, columns = X.columns, index = X.index.values.tolist())

## Random Forest Classifier

In [150]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
rf=RandomForestClassifier()

In [151]:
params = {'n_estimators': [10,100,200,400,500,1000], 'max_depth': [10,20,30,40,50,60,70,80,90,100,None], 'max_features': ['auto', 'sqrt','log2'], 'min_samples_leaf': [1,2,4], 'min_samples_split': [2,5,10], 'bootstrap': [True,False],'class_weight': ['balanced']}

In [152]:
cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=5, random_state=0)
rf_op = RandomizedSearchCV(rf, params, n_iter=20, cv=cv, scoring='f1', random_state=0, verbose=1, n_jobs=-1)

Only take 3 parameters for initial testing
Top 3 numerical features: cLogP, composition_B1,composition_A

In [154]:
X_train_RF = X_train[["cLogP_predicted", "composition_B1", "composition_A"]]
# Change X_train_RF here if input space is enlarged in later stages

In [155]:
rf_op.fit(X_train_RF,Y_train.values.ravel())

Fitting 20 folds for each of 20 candidates, totalling 400 fits


/Users/yimengjin/Desktop/Thesis/Thesis_Yimeng/venv/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/yimengjin/Desktop/Thesis/Thesis_Yimeng/venv/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/yimengjin/Desktop/Thesis/Thesis_Yimeng/venv/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past b

RandomizedSearchCV(cv=RepeatedStratifiedKFold(n_repeats=5, n_splits=4, random_state=0),
                   estimator=RandomForestClassifier(), n_iter=20, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'class_weight': ['balanced'],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [10, 100, 200, 400, 500,
                                                         1000]},
                   random_state=0, scoring='f1', verbose=1)

In [156]:
rf_op.best_params_

{'n_estimators': 400,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': None,
 'class_weight': 'balanced',
 'bootstrap': True}

In [157]:
rf = RandomForestClassifier(random_state=0, n_estimators=rf_op.best_params_['n_estimators'], min_samples_split=rf_op.best_params_['min_samples_split'], min_samples_leaf=rf_op.best_params_['min_samples_leaf'], max_features=rf_op.best_params_['max_features'], max_depth=rf_op.best_params_['max_depth'], class_weight = rf_op.best_params_['class_weight'],bootstrap=rf_op.best_params_['bootstrap'])

In [158]:
arr = cross_val_score(rf, X_train_RF, Y_train.values.ravel(), cv=cv)
arr

/Users/yimengjin/Desktop/Thesis/Thesis_Yimeng/venv/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/yimengjin/Desktop/Thesis/Thesis_Yimeng/venv/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/yimengjin/Desktop/Thesis/Thesis_Yimeng/venv/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past b

array([0.85      , 0.79487179, 0.82051282, 0.8974359 , 0.825     ,
       0.84615385, 0.84615385, 0.82051282, 0.875     , 0.92307692,
       0.76923077, 0.76923077, 0.825     , 0.8974359 , 0.76923077,
       0.82051282, 0.875     , 0.79487179, 0.87179487, 0.79487179])

In [159]:
np.mean(arr)

0.8342948717948719

In [160]:
rf.fit(X_train_RF,Y_train.values.ravel())

/Users/yimengjin/Desktop/Thesis/Thesis_Yimeng/venv/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomForestClassifier(class_weight='balanced', max_features='auto',
                       min_samples_leaf=2, min_samples_split=10,
                       n_estimators=400, random_state=0)

## Bayesian Optimisation

In [161]:
data_with_dummies

,composition_A,composition_B1,composition_B2,Number of blocks,dpn,A1,B1,C1,A2,B2,...,C4,cLogP_predicted,Category,type_A_Boc-AEAm,type_A_DMAEA,type_B1_PEAm,type_B2_None,type_C_HEAm,type_C_None,type_C_PEGA
0,0.5,0.30,0.00,1,100,0.3330,0.3330,0.334,0.0,0.0,...,0.0,0.67630,1,1,0,1,1,1,0,0
1,0.5,0.30,0.00,1,40,0.3325,0.3325,0.335,0.0,0.0,...,0.0,0.67630,1,1,0,1,1,1,0,0
2,0.5,0.30,0.00,1,20,0.3300,0.3350,0.335,0.0,0.0,...,0.0,0.67630,1,1,0,1,1,1,0,0
3,0.7,0.30,0.00,1,100,0.5000,0.5000,0.000,0.0,0.0,...,0.0,1.05270,0,1,0,1,1,0,1,0
4,0.7,0.30,0.00,1,40,0.5000,0.5000,0.000,0.0,0.0,...,0.0,1.05270,1,1,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,0.3,0.00,0.47,1,40,0.3325,0.3325,0.335,0.0,0.0,...,0.0,-1.33796,0,0,0,1,0,1,0,0
153,0.3,0.70,0.00,1,40,0.3325,0.3325,0.335,0.0,0.0,...,0.0,-0.16180,0,0,0,1,0,1,0,0
154,0.3,0.47,0.23,1,40,0.3325,0.3325,0.335,0.0,0.0,...,0.0,-0.45137,0,0,0,1,0,1,0,0
155,0.3,0.23,0.47,1,40,0.3325,0.3325,0.335,0.0,0.0,...,0.0,-0.75353,0,0,0,1,0,1,0,0


Top 3 numerical features: cLogP, composition_B1,composition_A

In [180]:
data_BO = X_train[["cLogP_predicted", "composition_B1", "composition_A"]]
data_BO

,cLogP_predicted,composition_B1,composition_A
0,0.572166,0.735233,-0.099386
1,0.572166,0.735233,-0.099386
2,0.572166,0.735233,-0.099386
3,0.869052,0.735233,1.148908
4,0.869052,0.735233,1.148908
...,...,...,...
152,-1.016587,-1.160195,-1.347681
153,-0.088888,3.262471,-1.347681
154,-0.317287,1.809309,-1.347681
155,-0.555616,0.292966,-1.347681


In [181]:
Y_train_BO = torch.tensor(data_with_dummies["Category"])
Y_train_BO = torch.reshape(Y_train_BO,(len(Y_train_BO),1))
Y_train_BO = Y_train_BO.float()
Y_train_BO

tensor([[1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
      

In [182]:
X_train_BO = torch.tensor(data_BO.values)
X_train_BO = X_train_BO.float()
best_y = 1.
X_train_BO, Y_train_BO, best_y

(tensor([[ 0.5722,  0.7352, -0.0994],
         [ 0.5722,  0.7352, -0.0994],
         [ 0.5722,  0.7352, -0.0994],
         [ 0.8691,  0.7352,  1.1489],
         [ 0.8691,  0.7352,  1.1489],
         [ 0.8691,  0.7352,  1.1489],
         [ 0.5722,  0.7352, -0.0994],
         [ 0.5722,  0.7352, -0.0994],
         [ 0.5722,  0.7352, -0.0994],
         [ 0.5722,  0.7352, -0.0994],
         [ 0.5722,  0.7352, -0.0994],
         [ 0.8691,  0.7352,  1.1489],
         [ 0.8691,  0.7352,  1.1489],
         [ 0.5722,  0.7352, -0.0994],
         [ 0.5722,  0.7352, -0.0994],
         [ 0.5722,  0.7352, -0.0994],
         [ 0.5722,  0.7352, -0.0994],
         [ 0.5722,  0.7352, -0.0994],
         [ 0.5722,  0.7352, -0.0994],
         [ 0.5722,  0.7352, -0.0994],
         [ 0.5722,  0.7352, -0.0994],
         [ 0.5722,  0.7352, -0.0994],
         [ 0.5722,  0.7352, -0.0994],
         [ 0.5722,  0.7352, -0.0994],
         [ 0.5722,  0.7352, -0.0994],
         [ 0.5722,  0.7352, -0.0994],
         [ 0

# Fit BO model
Physical constraints must be applied
### SingleTaskGP:
Only considers continuous inputs
### MixedSingleTaskGP:
Include discrete inputs and combines using a categorical kernel

In [183]:
model=SingleTaskGP(X_train_BO, Y_train_BO)
mll=ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_model(mll)

/Users/yimengjin/Desktop/Thesis/Thesis_Yimeng/venv/lib/python3.9/site-packages/botorch/models/gpytorch.py:113: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(


ExactMarginalLogLikelihood(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (noise_prior): GammaPrior()
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (model): SingleTaskGP(
    (likelihood): GaussianLikelihood(
      (noise_covar): HomoskedasticNoise(
        (noise_prior): GammaPrior()
        (raw_noise_constraint): GreaterThan(1.000E-04)
      )
    )
    (mean_module): ConstantMean()
    (covar_module): ScaleKernel(
      (base_kernel): MaternKernel(
        (lengthscale_prior): GammaPrior()
        (raw_lengthscale_constraint): Positive()
        (distance_module): Distance()
      )
      (outputscale_prior): GammaPrior()
      (raw_outputscale_constraint): Positive()
    )
  )
)

# Acquisition Function
## EI

In [184]:
EI = qExpectedImprovement(
    model = model,
    best_f = best_y
)

## UCB

In [192]:
UCB = qUpperConfidenceBound(
    model = model,
    beta = 0.6,
)

In [193]:
X_candidates,_ = optimize_acqf(
    acq_function = UCB,
    bounds = torch.tensor([[-5., 0., 0.],[5., 1., 1.]]),
    q = 20, # Number of suggested candidates
    num_restarts = 200,
    raw_samples = 512
)
X_candidates

tensor([[ 1.1977e+00,  6.3334e-01,  0.0000e+00],
        [ 4.0646e-02,  8.6015e-01,  6.2979e-01],
        [ 2.2469e+00,  5.8317e-01,  1.0367e-03],
        [-4.3468e+00,  5.7262e-01,  2.1308e-01],
        [ 2.4660e+00,  4.2374e-02,  1.0000e+00],
        [ 4.2969e+00,  7.8392e-01,  1.8611e-01],
        [-3.3532e+00,  5.5007e-01,  1.0551e-01],
        [-2.0821e+00,  8.2645e-01,  6.4503e-01],
        [ 1.0222e+00,  5.1165e-01,  0.0000e+00],
        [ 1.4848e+00,  6.5078e-01,  0.0000e+00],
        [ 3.9525e+00,  1.1453e-01,  5.0543e-01],
        [ 1.1581e+00,  8.8161e-01,  0.0000e+00],
        [-2.5345e+00,  6.5656e-01,  7.6524e-01],
        [ 1.0302e+00,  7.1135e-01,  0.0000e+00],
        [ 1.1161e+00,  6.4203e-01,  1.9762e-01],
        [-2.7038e+00,  6.4999e-01,  1.1681e-01],
        [-4.0169e+00,  5.1822e-01,  8.5543e-01],
        [ 3.3167e+00,  1.7212e-01,  9.7267e-01],
        [ 1.3784e+00,  8.2740e-01,  0.0000e+00],
        [ 1.3484e+00,  4.3447e-01,  0.0000e+00]])

## Substitute generated candidates to random forest classifier

In [194]:
Y_rf = rf.predict(X_candidates)
Y_rf

/Users/yimengjin/Desktop/Thesis/Thesis_Yimeng/venv/lib/python3.9/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1])